In [2]:
import cv2
from keras.preprocessing import image
import numpy as np
from skimage.transform import resize
import os
import re
import pickle
from keras.utils.np_utils import to_categorical

In [11]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.resnet50 import preprocess_input as ResNet_Pre
from tensorflow.keras.applications.densenet import preprocess_input as DenseNet_Pre
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as MobileNet_Pre

In [4]:
def vidToImages(videoFile, dim=(224,224), skipValue=6):

    cap = cv2.VideoCapture(videoFile)

    ImgArr = []

    while(cap.isOpened()):
        frameId = cap.get(cv2.CAP_PROP_POS_FRAMES)
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % skipValue == 0):
            resized = resize(frame, preserve_range=True, output_shape=dim).astype(int)
            ImgArr.append(resized)
    cap.release()
    
    return np.array(ImgArr)
    

In [5]:
def preprocess(Arr, process_func):
    img_array = []
    for img in Arr:
        t = image.img_to_array(img)
        t = np.expand_dims(t, axis=0)
        t = process_func(t)
        img_array.append(t)
    
    return np.vstack(img_array)

In [ ]:
model_ResNet = ResNet50(weights="imagenet", include_top=False, pooling='avg', input_shape=(224, 224, 3))
model_ResNet.summary()

In [ ]:
model_DenseNet = DenseNet121(weights="imagenet", include_top=False, pooling='avg', input_shape=(224, 224, 3))
model_DenseNet.summary()

In [ ]:
model_MobileNet = MobileNetV2(weights="imagenet", include_top=False, pooling='avg', input_shape=(224, 224, 3))
model_MobileNet.summary()

In [13]:
MainFolder = 'UCF11_updated_mpg'
dim = (224,224)
skipValue = 6
model = model_MobileNet
preProcess_func = MobileNet_Pre
filename_X = 'saveX_MobileNetV2.p'
filename_Y = 'saveY_MobileNetV2.p'


classes = {'basketball':0,
           'biking':1,
           'diving':2,
           'golf_swing':3,
           'horse_riding':4,
           'soccer_juggling':5,
           'swing':6,
           'tennis_swing':7,
           'trampoline_jumping':8,
           'volleyball_spiking':9,
           'walking':10}

total_classes = 11


X = []
Y = []


for path, dirs, files in os.walk(MainFolder):
    
    if re.search(r'^UCF11_updated_mpg\\.*\\Annotation', path) is not None:
        continue
    
    if re.search(r'^UCF11_updated_mpg\\.*\\.*', path) is not None:
        p = path.split('\\')[1]
        q = path.split('\\')[2]
        print("Extracting features from {} in {} folder...".format(q, p))
        
        for file in files:
            actual_path = os.path.join(path, file)
            t = vidToImages(actual_path, dim=dim, skipValue=skipValue)
            t = preprocess(t, preProcess_func)
            t = model.predict(t)
            X.append(t)
            Y.append(classes[p])


Y = to_categorical(Y, num_classes=total_classes)

print("Saving to a pickle file...")
with open(filename_X, 'wb') as fp:
    pickle.dump(X, fp)

with open(filename_Y, 'wb') as fp:
    pickle.dump(Y, fp)
    
print("Done.")

Extracting features from v_shooting_01 in basketball folder...
Extracting features from v_shooting_02 in basketball folder...
Extracting features from v_shooting_03 in basketball folder...
Extracting features from v_shooting_04 in basketball folder...
Extracting features from v_shooting_05 in basketball folder...
Extracting features from v_shooting_06 in basketball folder...
Extracting features from v_shooting_07 in basketball folder...
Extracting features from v_shooting_08 in basketball folder...
Extracting features from v_shooting_09 in basketball folder...
Extracting features from v_shooting_10 in basketball folder...
Extracting features from v_shooting_11 in basketball folder...
Extracting features from v_shooting_12 in basketball folder...
Extracting features from v_shooting_13 in basketball folder...
Extracting features from v_shooting_14 in basketball folder...
Extracting features from v_shooting_15 in basketball folder...
Extracting features from v_shooting_16 in basketball fo